In [1]:
import pandas as pd
import wands
from tqdm import tqdm
import os
import sys
sys.path.append('..')

import utils

In [2]:
dataset = 'wands'
model_folder = 'text-exp0307'
model_id = 'text-embedding-large-exp-03-07'

model_folder = 'text-004'
model_id = 'text-embedding-004'

In [3]:
q = pd.read_parquet(f'{dataset}/queries.parquet')
p = pd.read_parquet(f'{dataset}/products.parquet')


In [4]:
project_id = 'gemma-test-deployment'

In [5]:
q.head()

,query,query_class
query_id,,
0,salon chair,Massage Chairs
1,smart coffee table,Coffee & Cocktail Tables
2,dinosaur,Kids Wall Décor
3,turquoise pillows,Accent Pillows
4,chair and a half recliner,Recliners


In [6]:
queries = q
queries['embeddings']= list(utils.get_gemini_embeddings(list(queries['query'].values), 
                                                   model=model_id,
                                                   project_id = project_id,
                                                   location = "us-central1",
                                                   task_type = 'RETRIEVAL_QUERY', 
                                                  n_jobs=-1))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   24.6s finished


In [19]:
queries.to_parquet(f'{model_folder}/{dataset}-queries.parquet')

In [18]:
queries.head()

,query,query_class,embeddings
query_id,,,
0,salon chair,Massage Chairs,"[-0.02360038086771965, -0.05621311441063881, 0..."
1,smart coffee table,Coffee & Cocktail Tables,"[-0.012133768759667873, -0.03541422635316849, ..."
2,dinosaur,Kids Wall Décor,"[-0.017517710104584694, -0.06345050781965256, ..."
3,turquoise pillows,Accent Pillows,"[0.02777988277375698, -0.04077966511249542, -0..."
4,chair and a half recliner,Recliners,"[0.00022231682669371367, -0.05051223933696747,..."


In [11]:
products = p

fname = f'{model_folder}/{dataset}-products.parquet'

split_size = 10000
for i in range(0,len(products),split_size):
    split_fname = f'{fname}-split-{i:07d}'
    product_split = products.iloc[i:i+split_size].copy()
    print (split_fname, product_split.shape, flush=True)
    if os.path.isfile(split_fname):
        continue
    product_split['embeddings']= list(utils.get_gemini_embeddings(list(product_split['product'].values), 
                                                   model=model_id,
                                                   project_id = project_id,
                                                   location = "us-central1",
                                                   task_type = 'RETRIEVAL_DOCUMENT', 
                                                  n_jobs=-1))

    product_split.to_parquet(split_fname)

text-004/wands-products.parquet-split-0000000 (10000, 1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 1122 tasks      | elapsed:   43.9s


..........

[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2322 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 3072 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:  2.7min


..

...

[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    5.5s


..

[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 1122 tasks      | elapsed:   43.4s


..........

[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2322 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2994 out of 2994 | elapsed:  2.2min finished


## collate files

In [12]:
from glob import glob
import pandas as pd

In [13]:
sorted(glob(f'{model_folder}/{dataset}-products.parquet-split-*'))

['text-004/wands-products.parquet-split-0000000',
 'text-004/wands-products.parquet-split-0010000',
 'text-004/wands-products.parquet-split-0020000',
 'text-004/wands-products.parquet-split-0030000',
 'text-004/wands-products.parquet-split-0040000']

In [15]:
all_products = pd.concat([pd.read_parquet(i) for i in sorted(glob(f'{model_folder}/{dataset}-products.parquet-split-*'))])

In [16]:
all_products.shape

(42994, 2)

In [17]:
all_products.to_parquet(f'{model_folder}/{dataset}-products.parquet')